In [0]:
%sql
select * from silver_ehr.patients limit 10;

## **Total COVID Positive Patients**

In [0]:

%sql
CREATE OR REPLACE TABLE gold_ehr.covid_positive_patients AS
WITH covid_patients AS (
    SELECT
        patient_id,
        MIN(start_date)                              AS first_covid_date,
        MIN(condition_year)                          AS diagnosis_year,
        MIN(condition_month)                         AS diagnosis_month
    FROM silver_ehr.conditions
    WHERE snomed_code = '840539006'               -- Confirmed COVID-19 only
    GROUP BY patient_id
),

demographics AS (
    SELECT
        cp.diagnosis_year,
        cp.diagnosis_month,
        p.gender,
        p.race,
        p.ethnicity,
        p.age_group,
        p.state,
        COUNT(DISTINCT cp.patient_id)                AS covid_positive_patients
    FROM covid_patients cp
    JOIN silver_ehr.patients p
        ON cp.patient_id = p.patient_id
    GROUP BY
        cp.diagnosis_year,
        cp.diagnosis_month,
        p.gender,
        p.race,
        p.ethnicity,
        p.age_group,
        p.state
)

SELECT
    diagnosis_year,
    diagnosis_month,
    gender,
    race,
    ethnicity,
    age_group,
    state,
    covid_positive_patients,
    SUM(covid_positive_patients) OVER ()             AS grand_total_covid_patients,
    SUM(covid_positive_patients) OVER (
        PARTITION BY diagnosis_year
        ORDER BY diagnosis_month
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    )                                                AS cumulative_count_in_year
FROM demographics
ORDER BY
    diagnosis_year,
    diagnosis_month;

 **Hospitalization Rate**

In [0]:
%sql
CREATE OR REPLACE TABLE gold_ehr.covid_hospitalization_rate AS
WITH covid_dx AS (
    SELECT
        patient_id,
        MIN(start_date)                              AS first_covid_date
    FROM silver_ehr.conditions
    WHERE snomed_code = '840539006'
    GROUP BY patient_id
),

hospitalised AS (
    SELECT DISTINCT
        e.patient_id
    FROM silver_ehr.encounters e
    JOIN covid_dx cd
        ON  e.patient_id    = cd.patient_id
        AND e.encounter_date >= cd.first_covid_date
    WHERE e.encounter_class IN ('inpatient', 'emergency')
),
summary AS (
    SELECT
        p.age_group,
        p.gender,
        p.race,
        p.ethnicity,
        COUNT(DISTINCT cd.patient_id)                AS total_covid_patients,
        COUNT(DISTINCT h.patient_id)                 AS hospitalised_patients
    FROM covid_dx cd
    JOIN silver_ehr.patients p
        ON cd.patient_id = p.patient_id
    LEFT JOIN hospitalised h
        ON cd.patient_id = h.patient_id
    GROUP BY
        p.age_group,
        p.gender,
        p.race,
        p.ethnicity
)

SELECT
    age_group,
    gender,
    race,
    ethnicity,
    total_covid_patients,
    hospitalised_patients,
    ROUND(
        100.0 * hospitalised_patients / NULLIF(total_covid_patients, 0), 2
    )                                                AS hospitalization_rate_pct,
    -- Overall benchmark across all groups
    ROUND(
        100.0 * SUM(hospitalised_patients) OVER ()
              / NULLIF(SUM(total_covid_patients) OVER (), 0), 2
    )                                                AS overall_hospitalization_rate_pct
FROM summary
ORDER BY
    hospitalization_rate_pct DESC , age_group DESC;

**### Mortality Rate**

In [0]:
 %sql 
WITH covid_patients AS (
  SELECT DISTINCT
    p.patient_id,
    p.race,
    p.gender,
    p.ethnicity,
    COALESCE(
      p.age_group,
      CASE
        WHEN (p.death_year - p.birth_year) < 18 THEN '0-17'
        WHEN (p.death_year - p.birth_year) < 35 THEN '18-34'
        WHEN (p.death_year - p.birth_year) < 50 THEN '35-49'
        WHEN (p.death_year - p.birth_year) < 65 THEN '50-64'
        ELSE '65+'
      END
    )                                             AS age_group,
    p.is_deceased
  FROM silver_ehr.conditions c
  JOIN silver_ehr.patients p ON c.patient_id = p.patient_id
  WHERE c.snomed_code = '840539006'
)

SELECT
  gender,
  race,
  age_group,
  ethnicity,
  COUNT(CASE WHEN is_deceased = TRUE THEN 1 END)  AS deceased_covid,
  COUNT(*)                                         AS total_covid_patients,
  ROUND(
    COUNT(CASE WHEN is_deceased = TRUE THEN 1 END) * 100.0
    / NULLIF(COUNT(*), 0), 2
  )                                                AS mortality_rate_pct
FROM covid_patients
GROUP BY gender, age_group, race, ethnicity
ORDER BY mortality_rate_pct DESC;

 **Average Length of Stay**

In [0]:
%sql 
CREATE OR REPLACE TABLE gold_ehr.covid_avg_los AS
WITH covid_inpatient AS (
  SELECT
    p.patient_id,
    p.gender,
    COALESCE(p.age_group,
      CASE
        WHEN (p.death_year - p.birth_year) < 18 THEN '0-17'
        WHEN (p.death_year - p.birth_year) < 35 THEN '18-34'
        WHEN (p.death_year - p.birth_year) < 50 THEN '35-49'
        WHEN (p.death_year - p.birth_year) < 65 THEN '50-64'
        ELSE '65+'
      END
    )                           AS age_group,
    e.encounter_class,         
    e.duration_days             AS los_days
  FROM silver_ehr.conditions  c
  JOIN silver_ehr.patients    p ON c.patient_id  = p.patient_id
  JOIN silver_ehr.encounters  e ON c.patient_id  = e.patient_id
                              AND e.encounter_date >= c.start_date
  WHERE c.snomed_code    = '840539006'
    AND e.duration_days   > 0
),

patient_los AS (
  SELECT
    patient_id,
    gender,
    age_group,
    encounter_class,           
    AVG(los_days)               AS avg_los_days
  FROM covid_inpatient
  GROUP BY patient_id, gender, age_group, encounter_class
)

SELECT
  encounter_class,
  gender,
  age_group,
  COUNT(*)                      AS patients_count,
  ROUND(AVG(avg_los_days), 2)   AS overall_avg_los_days,
  ROUND(MIN(avg_los_days), 2)   AS min_los_days,
  ROUND(MAX(avg_los_days), 2)   AS max_los_days
FROM patient_los
GROUP BY encounter_class, gender, age_group
ORDER BY encounter_class, overall_avg_los_days DESC;

Vaccinated vs Non-Vaccinated Hospitalization

In [0]:
%sql 
CREATE OR REPLACE TABLE gold_ehr.covid_vax_hosp_comparison AS
WITH covid_patients AS (
  SELECT DISTINCT
    p.patient_id,
    p.gender,
    p.age_group,
    p.race
  FROM silver_ehr.conditions c
  JOIN silver_ehr.patients p ON c.patient_id = p.patient_id
  WHERE c.snomed_code = '840539006'
),

vax_status AS (
  SELECT
    cp.patient_id,
    cp.gender,
    cp.age_group,
    cp.race,
    CASE
      WHEN COUNT(DISTINCT i.DATE) > 0 THEN 'Vaccinated'
      ELSE 'Non-Vaccinated'
    END AS vax_status
  FROM covid_patients cp
  LEFT JOIN silver_ehr.immunizations i ON cp.patient_id = i.PATIENT
  GROUP BY cp.patient_id, cp.gender, cp.age_group, cp.race
),

hospitalizations AS (
  SELECT
    vs.*,
    CASE WHEN e.encounter_class = 'inpatient' THEN 1 ELSE 0 END AS is_hospitalized
  FROM vax_status vs
  LEFT JOIN silver_ehr.encounters e ON vs.patient_id = e.patient_id
    AND e.encounter_class = 'inpatient'
)

SELECT
  gender,
  age_group,
  race,
  vax_status,
  COUNT(*) AS total_covid_patients,
  SUM(is_hospitalized) AS hospitalized_count,
  ROUND(SUM(is_hospitalized) * 100.0 / COUNT(*), 2) AS hosp_rate_pct
FROM hospitalizations
GROUP BY gender, age_group, race, vax_status
ORDER BY
  gender,
  age_group,
  race,
  CASE vax_status WHEN 'Non-Vaccinated' THEN 1 ELSE 2 END,
  hosp_rate_pct DESC;

Common Comorbidities in COVID Patients

In [0]:
%sql  
CREATE OR REPLACE TABLE gold_ehr.covid_comorbidities AS 
WITH covid_dx AS (
    SELECT
        patient_id,
        MIN(start_date)                           AS first_covid_date
    FROM silver_ehr.conditions
    WHERE snomed_code = '840539006'
    GROUP BY patient_id
),
total_covid_patients AS (
    SELECT COUNT(DISTINCT patient_id)             AS n
    FROM covid_dx
),
comorbidities AS (
    SELECT
        c.snomed_code                             AS condition_code,
        c.description                             AS condition_name,
        COUNT(DISTINCT c.patient_id)              AS affected_patients
    FROM silver_ehr.conditions c
    JOIN covid_dx cd
        ON c.patient_id = cd.patient_id
    WHERE c.start_date < cd.first_covid_date
      AND c.snomed_code <> '840539006'
    GROUP BY c.snomed_code, c.description
)
SELECT
    co.condition_code,
    co.condition_name,
    co.affected_patients,
    tcp.n                                         AS total_covid_patients,
    ROUND(
        100.0 * co.affected_patients / tcp.n, 2
    )                                             AS prevalence_pct,
    RANK() OVER (
        ORDER BY co.affected_patients DESC
    )                                             AS comorbidity_rank
FROM comorbidities co
CROSS JOIN total_covid_patients tcp
ORDER BY comorbidity_rank;

In [0]:
%sql
select count (*) from gold_ehr.covid_vax_hosp_comparison